# Football Data Import and Table Relationships in PostgreSQL

This project showcases a Python script that automates the process of importing football data from CSV files into PostgreSQL tables. The script establishes relationships between the tables, ensuring data consistency and enabling efficient querying and analysis.

## Key Steps

1. **Data Reading**: The script utilizes the powerful pandas library to read the football data from CSV files and store it in pandas DataFrames.
2. **Database Connection**: Establish a secure and reliable connection to the PostgreSQL database using psycopg2, a popular PostgreSQL adapter for Python.
3. **Table Creation**: SQL statements are executed to create tables for match results, shootouts, and goalscorers. This ensures a structured and organized representation of the data.
4. **Data Insertion**: The script efficiently inserts the data from DataFrames into the respective PostgreSQL tables, leveraging the SQLAlchemy library for seamless interaction with the database.
5. **Table Relationship Update**: The script updates the shootouts and goalscorers tables with the corresponding result_id from the results table based on matching date, home team, and away team. This establishes relationships between the tables, allowing for meaningful data analysis.
6. **Referential Integrity**: Foreign key constraints are added to the shootouts and goalscorers tables, ensuring referential integrity with the results table. This enhances data integrity and consistency within the database.

The Football Data Import and Table Relationships project aims to simplify the data engineering process, making it easier to import, store, and query football data in a PostgreSQL database. By automating these tasks, analysts and data scientists can focus on extracting valuable insights from the data without worrying about the complexities of data management.

Feel free to customize and refine this introduction according to your project's specific details and requirements.

### Import libraries

In [77]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

### Function to create a database if needed and connect to it

In [78]:
def create_connection(host, port, database, user, password):
    # Connect to the default PostgreSQL database (e.g., "postgres")
    default_conn = psycopg2.connect(
        host=host,
        port=port,
        database='postgres',
        user=user,
        password=password
    )
    
    default_conn.autocommit = True
    default_cursor = default_conn.cursor()
    
    # Check if the specified database exists
    default_cursor.execute("SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s;", (database,))
    exists = default_cursor.fetchone()
    
    if not exists:
        # Create the specified database if it doesn't exist
        default_cursor.execute(f"CREATE DATABASE {database};")
    
    default_cursor.close()
    default_conn.close()
    
    # Connect to the specified database
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    
    return conn

### Function to execute a SQL query and commit changes

In [79]:
def execute_query(conn, query):
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()

### Function to insert DataFrame data into a PostgreSQL table

In [80]:
def insert_data(conn, df, table_name):
    engine = create_engine(f'postgresql://{conn.info.user}:{conn.info.password}@{conn.info.host}:{conn.info.port}/{conn.info.dbname}')
    df.to_sql(table_name, engine, if_exists='append', index=False)

### Queries to create tables for match results, shootouts, and goalscorers

In [81]:
create_results_table_query = """
    CREATE TABLE IF NOT EXISTS results (
        id SERIAL PRIMARY KEY,
        date DATE,
        home_team TEXT,
        away_team TEXT,
        home_score INTEGER,
        away_score INTEGER,
        tournament TEXT,
        city TEXT,
        country TEXT,
        neutral BOOLEAN
    );
"""

create_shootouts_table_query = """
    CREATE TABLE IF NOT EXISTS shootouts (
        id SERIAL PRIMARY KEY,
        result_id INTEGER,
        date DATE,
        home_team TEXT,
        away_team TEXT,
        winner TEXT
    );
"""

create_goalscorers_table_query = """
    CREATE TABLE IF NOT EXISTS goalscorers (
        id SERIAL PRIMARY KEY,
        result_id INTEGER,
        date DATE,
        home_team TEXT,
        away_team TEXT,
        team TEXT,
        scorer TEXT,
        minute INTEGER,
        own_goal BOOLEAN,
        penalty BOOLEAN
    );
"""

### Queries to update shootouts and goalscorers tables with result_id from results table

In [82]:
update_shootouts_query = """
    UPDATE shootouts
    SET result_id = (
        SELECT id
        FROM results
        WHERE shootouts.date = results.date
            AND shootouts.home_team = results.home_team
            AND shootouts.away_team = results.away_team
    )
    WHERE result_id IS NULL;
"""

update_goalscorers_query = """
    UPDATE goalscorers
    SET result_id = (
        SELECT id
        FROM results
        WHERE goalscorers.date = results.date
            AND goalscorers.home_team = results.home_team
            AND goalscorers.away_team = results.away_team
    )
    WHERE result_id IS NULL;
"""

### Queries to add foreign key constraints to shootouts and goalscorers tables

In [83]:
add_fk_constraint_shootouts_query = """
    ALTER TABLE shootouts
    ADD CONSTRAINT fk_result_id
    FOREIGN KEY (result_id)
    REFERENCES results(id);
"""

add_fk_constraint_goalscorers_query = """
    ALTER TABLE goalscorers
    ADD CONSTRAINT fk_result_id
    FOREIGN KEY (result_id)
    REFERENCES results(id);
"""

### Read the CSV files

In [84]:
results_df = pd.read_csv('datasets/results.csv')
shootouts_df = pd.read_csv('datasets/shootouts.csv')
goalscorers_df = pd.read_csv('datasets/goalscorers.csv')

### Create database connection

In [85]:
conn = create_connection(host='localhost', port='5432', database='football_data', user='postgres', password='root*')

### Create tables

In [86]:
execute_query(conn, create_results_table_query)
execute_query(conn, create_shootouts_table_query)
execute_query(conn, create_goalscorers_table_query)

### Insert data into tables

In [87]:
insert_data(conn, results_df, 'results')
insert_data(conn, shootouts_df, 'shootouts')
insert_data(conn, goalscorers_df, 'goalscorers')

### Update shootouts table with result_id from results table

In [88]:
execute_query(conn, update_shootouts_query)

### Update goalscorers table with result_id from results table

In [89]:
execute_query(conn, update_goalscorers_query)

### Add foreign key constraints

In [90]:
execute_query(conn, add_fk_constraint_shootouts_query)
execute_query(conn, add_fk_constraint_goalscorers_query)

### Close the connection

In [91]:
conn.close()